<h1>Algoritmo Minimax</h1><br>
<b><i>Minimax es un algoritmo que se basa en <code>backtracking</code> para calcular todos los posibles escenarios en un espacio de busqueda. El cual en este caso serían las decisiones que los jugadores pueden tomar.</i></b><br><br>
Como explicamos en el <a href="Trabajo_Parcial_Complejidad.ipynb">informe principal</a>, este algoritmo sería usado para la toma de decisiones de los jugadores en Quoridor. Sin embargo, esta es una demostración del algoritmo utilizando Tres en Linea.


<h2>Funciones Básicas del juego</h2>

<h3>Funciones Auxiliares</h3>

In [6]:
def valid_move(board, pos):
    if len(pos) == 2:
        if not(0 <= pos[0] and pos[0] < 3 and 0 <= pos[1] and pos[1] < 3):
            print(pos, "Out of bounds"); return False
        if not(board[pos[0]][pos[1]] == ' '):
            print(pos, "Is occupied"); return False
        return True

def print_winner(winner):
    print(f"Winner is {'X' if winner == 1 else 'O'}!\n")

def pb(board): #print_board
    for i in board: print(i)

<h3>Validar ganadores</h3><br>
Es necesario tomar en cuenta que se está tomando los valores de $-1$ y $1$ para representar cuál de los jugadores es el que gana en cada caso. <br>De no ser ninguno retorna $0$.<br>
También prestar atención a la funcion <code>check_winner()</code>. Su nombre explica su función pero esta retornará una tupla, cuyo <code>primer componente</code> será un bool que definirá si el juego termina o no, ya sea porque se empató o uno de los jugadores ganó y su <code>segunda componente</code> será un valor: $-1$, $0$ o $1$, que representará el estado actual del movimiento. Servirá para identificar quién fue el que ganó si la primera componente retorna <code>True</code> y para ver si la 'AI' se acerca al resultado deseado. 

In [10]:
def check_rows(board): 
    for row in board:
        if not ' ' in row:
            if not 'X' in row: return -1
            if not 'O' in row: return 1
    return 0
def check_cols(board):
    for i in range(3):
        col = [row[i] for row in board]
        if not ' ' in col:
            if not 'X' in col: return -1
            if not 'O' in col: return 1
    return 0
def check_diagonals(board):
    d1 = []
    d2 = []
    for i in range(3):
        d1.append(board[i][i])
        d2.append(board[2-i][i])
    if not ' ' in d1:
            if not 'X' in d1: return -1
            if not 'O' in d1: return 1
    if not ' ' in d2:
            if not 'X' in d2: return -1
            if not 'O' in d2: return 1
    return 0    
def check_winner(board): ### True continues
    ###  1 = 'X'
    ### -1 = 'O'
    winner = check_rows(board)
    if winner != 0: return False, winner
    winner = check_cols(board)
    if winner != 0: return False, winner
    winner = check_diagonals(board)
    if winner != 0: return False, winner
    else: #draw or continue
        ### board is full
        if any(' ' in row for row in board): return True, 0 ## continue
        else: return False, 0 ## stops

<h2>El algoritmo principal</h2><br>
Minimax es un algoritmo que se basa en <code>un solo puntaje</code>, el cual los jugadores tratan de <b>maximizar o minimizar</b> de acuerdo a su conveniencia. En el caso de nuestra IA (la 'O'), el valor que usamos para definir su victoria es el $-1$, por lo tanto le conviene que el puntaje general disminuya ya sea a $0$ o a $-1$ y minimax iterará por todos los posibles movimientos y luego los movimientos posibles luego de cada uno de esos movimientos hasta que se pierda, gane o empate y de acuerdo a eso, volver al estado actual y retornar el puntaje del juego.<br>
<br><b>Puntaje el cual</b> será tomado en cuenta por la función <code>get_best_move()</code>, que retornará la posición en la que la 'IA' deberá marcar. Se iterará por toda la tabla y se usará el concepto de <code>backtracking</code> para seguir el juego y luego eliminar los cambios realizados, luego de calcular el valor de <code>Minimax</code> en ese movimiento. Luego de haber calculado todos los posibles puntajes en cada movimiento, se elegirá (en este caso) el menor. 

In [8]:
def minimax(board, maximizing): #### status (bool(continue/stop), int([-1,1]: W,T,L))
    status = check_winner(board)
    if status[0] == False:
        return status[1] ## W/L/T case
    if maximizing:
        best_score = float('-inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'X'
                    best_score = max(minimax(board,False), best_score)
                    board[i][j] = ' '    
        return best_score

    else:
        best_score = float('inf')           
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'O'
                    best_score = min(minimax(board, True), best_score)
                    board[i][j] = ' ' 
        return best_score

def get_best_move(board): ### for O
    best_score = float('inf')
    new_move = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                board[i][j] = 'O'
                s = minimax(board, True)
                if s < best_score:
                    best_score = s
                    new_move = [i,j]                
                board[i][j] = ' '
    return new_move

<h3>Iniciar el juego</h3><br>
En las siguientes lineas lo único que pasa es la inicialización de las variables y un while donde se iterará mientras el juego no haya empatado ni ganado. Y dentro se pedirá el input del jugador o caso contrario la mejor posición que puede tomar el oponente de acuerdo a lo que indicaron las funciones previas. <br><b> En el output se demuestra el funcionamiento </b>

In [9]:
from random import getrandbits
board = [[' ',' ',' '],
         [' ',' ',' '],
         [' ',' ',' ']]

ai = [-1,-1]
player = [-1,-1]
turn = bool(getrandbits(1)) ### turn 1 ai turn -1 player

while check_winner(board)[0]:
    print()
    if turn: 
        print("Player's turn:", end = " ")
        player = [int(x) for x in input().split()]
        if valid_move(board,player): board[player[0]][player[1]] = 'X'
        else: turn = not turn
    else: 
        print("Ai's turn:", end = "\n")
        ai = get_best_move(board)
        if valid_move(board,ai): board[ai[0]][ai[1]] = 'O'
        else: turn = not turn

    pb(board)
    w = check_winner(board)
    if w[0] == False: 
        if w[1] != 0: print_winner(w[1])
        else: print("Draw!")
    turn = not turn


Ai's turn:
['O', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Player's turn: 1 2
['O', ' ', ' ']
[' ', ' ', 'X']
[' ', ' ', ' ']

Ai's turn:
['O', ' ', 'O']
[' ', ' ', 'X']
[' ', ' ', ' ']

Player's turn: 0 1
['O', 'X', 'O']
[' ', ' ', 'X']
[' ', ' ', ' ']

Ai's turn:
['O', 'X', 'O']
[' ', 'O', 'X']
[' ', ' ', ' ']

Player's turn: 2 2
['O', 'X', 'O']
[' ', 'O', 'X']
[' ', ' ', 'X']

Ai's turn:
['O', 'X', 'O']
[' ', 'O', 'X']
['O', ' ', 'X']
Winner is O!



<h2>Conclusiones</h2><br>
    
<code>El espacio de busqueda de Minimax</code>en este caso no se aprecia tanto. Se podría optimizar bastante salteando el cálculo cuándo la 'AI' tiene el primer turno y elegir una de las esquinas o el centro, puesto que cuando las 9 casillas están libres.<br>
<br><b>Por otro lado</b>, la complejidad de minimax usado en otros casos como Quoridor, presentaría un problema, porque vendría a ser $O(b^m)$ siendo $m$ las posibles decisiones y $b$ la cantidad de veces que se iteraría. Entonces deberiamos usar un concepto conocido como <code>Alpha Beta prunning</code> explicado mejor en el <a href="Trabajo_Parcial_Complejidad.ipynb">informe principal</a>.

<br>
En Quoridor el espacio de búsqueda será mucho mayor y tendremos la posibilidad de tomar 2 decisiones, movernos o colocar una barrera, por lo tanto necesitaremos hacer lo más eficiente el algoritmo de minimax. 
